# Using Tweepy to sniff out Twitter bots

### Load python tools

In [1]:
import pandas as pd
import numpy as np
import tweepy
import json

### Get authenticated

In [2]:
consumer_key="0EtWySJEum7PpQ7ubreQgLwH5"
consumer_secret="6DPNvqyx44VlJWL3AAdNj53hmkiNAh0uGrvjfVH0ixmyLLqQe7"
access_token="14790314-YvDi7tRrBMQNqPo2BEKqAx7n7lo8fBnA3InySZcg9"
access_token_secret="iKL0A2Hl5oBlbV3m0ffYGWsjK4EVnS9VxvjFRPCCfW5XZ"

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [4]:
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [5]:
rt = api.retweets('1316821974115479552')

In [6]:
ID = 1316821974115479552

In [7]:
count = 100

### Search results for "VoteNoOnHolly"

In [8]:
results = api.search(q='VoteNoOnHolly', count=100)

### Get our list of retweets the @VoteNoOnHolly tweet

In [9]:
retweets_list = api.retweets(ID, count)

### Create a list of retweeters by screen_name

In [10]:
rters = []

for retweet in retweets_list: 
    rters.append(retweet.user.screen_name)

### Get user information for each retweeter

In [11]:
df_users = []

for r in rters:
    screen_name = r
    user = api.get_user(screen_name)
    df_users.append(user._json)

### Users into a dataframe

In [12]:
df = pd.DataFrame(df_users)

In [13]:
dfslim = pd.DataFrame(df[['name', 'screen_name', 'location', 'description', 'followers_count','friends_count', \
    'created_at', 'favourites_count', 'statuses_count','profile_image_url',]])

In [14]:
dfslim['created'] = pd.to_datetime(dfslim['created_at']).dt.date

In [15]:
dfslim = dfslim.drop(['created_at'], axis=1)

In [16]:
dfslim.head()

,name,screen_name,location,description,followers_count,friends_count,favourites_count,statuses_count,profile_image_url,created
0,Дмитрий,KNNMgw9eUKOaFMf,,,4,528,69,276,http://pbs.twimg.com/profile_images/1239844866...,2020-03-17
1,aleksandr,aleksan32030133,,,3,219,85,202,http://pbs.twimg.com/profile_images/1306827552...,2020-09-18
2,Norayr,Norayr09895905,,Հայաստանը իմն է,2,184,55,141,http://pbs.twimg.com/profile_images/1276479305...,2020-06-26
3,Сергеи Иванов,qOb28mnrRnuR7ry,,Я добрый!!,4,780,64,258,http://pbs.twimg.com/profile_images/1240943010...,2020-03-19
4,Svetlana Khomyak,KhomyakSvetlana,,Счатливая жена и мама 2 прекрасных детей,6,423,121,292,http://pbs.twimg.com/profile_images/1286315340...,2020-07-23


---

In [ ]:
for user in tweepy.Cursor(api.friends, screen_name="qOb28mnrRnuR7ry").items():
    print('friend: ' + user.screen_name)

Rate limit reached. Sleeping for: 615


In [ ]:
for user in tweepy.Cursor(api.followers, screen_name="qOb28mnrRnuR7ry").items():
    print('follower: ' + user.screen_name)

---

### (Quietly) get all the photos!

In [ ]:
dfslim['profile_image_url'] = dfslim['profile_image_url'].str.replace('_normal', '')

In [ ]:
for photo in dfslim.profile_image_url:
    !wget -c -q '{photo}' -P 'images/' 

---